In [238]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [239]:
import matplotlib
import matplotlib.pyplot as plt
import glob
import math
import numpy as np
import pandas as pd
import numba
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [240]:
import trackml
from trackml.dataset import load_event, load_dataset

In [241]:
# @numba.vectorize
def haveSimilarCurvature(x1,y1,x2,y2,x3,y3, ptmin, region_origin_radius, phiCut, hardPtCut) :
    region_origin_x = 0
    region_origin_y = 0

    distance_13_squared = (x1 - x3)*(x1 - x3) + (y1 - y3)*(y1 - y3)
    tan_12_13_half_mul_distance_13_squared = abs(y1 * (x2 - x3) + y2 * (x3 - x1) + y3 * (x1 - x2))
    # high pt : just straight
    straight = tan_12_13_half_mul_distance_13_squared * ptmin <= 1.0e-4*distance_13_squared
    def ifStraight() :
        distance_3_beamspot_squared = (x3-region_origin_x) * (x3-region_origin_x) + (y3-region_origin_y) * (y3-region_origin_y)
        dot_bs3_13 = ((x1 - x3)*( region_origin_x - x3) + (y1 - y3) * (region_origin_y-y3))
        proj_bs3_on_13_squared = dot_bs3_13*dot_bs3_13/distance_13_squared
        distance_13_beamspot_squared  = distance_3_beamspot_squared -  proj_bs3_on_13_squared
        return distance_13_beamspot_squared < (region_origin_radius+phiCut)*(region_origin_radius+phiCut)
    
    def standard() :
        # 87 cm/GeV = 1/(3.8T * 0.3)
        # 165 cm/GeV = 1/(2T * 0.3)
    
        # take less than radius given by the hardPtCut and reject everything below
        minRadius = hardPtCut*1650 #  // FIXME move out and use real MagField
    
        det = (x1 - x2) * (y2 - y3) - (x2 - x3) * (y1 - y2)
    
        offset = x2*x2 + y2*y2
    
        bc = (x1*x1 + y1*y1 - offset)*0.5
    
        cd = (offset - x3*x3 - y3*y3)*0.5
    
    
    
        idet = 1./ det;
    
        x_center = (bc * (y2 - y3) - cd * (y1 - y2)) * idet
        y_center = (cd * (x1 - x2) - bc * (x2 - x3)) * idet
    
        radius = np.sqrt((x2 - x_center)*(x2 - x_center) + (y2 - y_center)*(y2 - y_center))
    
        def domore() :
            centers_distance_squared = (x_center - region_origin_x)*(x_center - region_origin_x) + (y_center - region_origin_y)*(y_center - region_origin_y)
            region_origin_radius_plus_tolerance = region_origin_radius + phiCut
            minimumOfIntersectionRange = (radius - region_origin_radius_plus_tolerance)*(radius - region_origin_radius_plus_tolerance)
            ok = centers_distance_squared >= minimumOfIntersectionRange
            return ok & (centers_distance_squared <= (radius + region_origin_radius_plus_tolerance)*(radius + region_origin_radius_plus_tolerance))


        return (radius > minRadius) & domore()
    
    return np.where(straight,ifStraight(),standard())
    # return standard()

In [242]:
curvcut = 0.5/(1650*0.3) # 1/(1650*pt)
rl1 = 32
rl2 = 72
rl3 = 115
rl4 = 171
def deltaphi(a,b,ch=curvcut) :
    d = b-a
    cd = ch*d
    ca = ch*a
    return cd*(1.+0.5*ca*(ca+cd)+cd*cd*0.1667)

dp12 = deltaphi(rl1,rl2)
dp23 = deltaphi(rl2,rl3)
dp34 = deltaphi(rl3,rl4)
print dp12,dp23,dp34
mpcor = 0.01

0.0404625262934 0.0436314721719 0.0571633024328


In [243]:
@numba.vectorize(nopython=True)
def areAlignedRZ(ri, zi, r1, z1, ro, zo, ptmin, thetaCut) :
    radius_diff = abs(r1 - ro)
    distance_13_squared = radius_diff*radius_diff + (z1 - zo)*(z1 - zo)
    
    pMin = ptmin*np.sqrt(distance_13_squared) #this needs to be divided by radius_diff later
    
    tan_12_13_half_mul_distance_13_squared = abs(z1 * (ri - ro) + zi * (ro - r1) + zo * (r1 - ri)) 
    return tan_12_13_half_mul_distance_13_squared * pMin <= thetaCut * distance_13_squared * radius_diff

In [244]:
def doublets(hits,l1,l2,cut=0.1) :
    nd=0
    for h1 in l1.itertuples() :
        phi = h1.phi
        hh = l2['phi'].searchsorted([phi-cut,phi+cut])
        hits.loc[h1.Index,'up0'] = hh[0]
        hits.loc[h1.Index,'up1'] = hh[1]
        nd += hh[1]-hh[0]
    return nd

In [245]:
#file = '/Users/innocent/data/trackML_minitrain.zip'
#file = '/Users/innocent/data/train_100_events'
# maxEv = 2


def runit(file = '/Users/innocent/data/trackML_minitrain.zip', maxEv=2):
  evn=0  
  for event_id, hits, cells, particles, truth in load_dataset(file):
    evn+=1
    if (evn>maxEv) : break
    print event_id, len(hits)
    hits = pd.merge(hits,truth,on='hit_id')
    hits['phi'] = np.arctan2(hits['y'],hits['x'])
    hits['r'] = np.sqrt(hits['y']*hits['y']+hits['x']*hits['x'])
    hits['tpt'] = np.sqrt(hits['tpx']*hits['tpx']+hits['tpy']*hits['tpy'])
    hits['up0'] =0
    hits['up1'] =0
    hits.sort_values(by=['phi'],inplace=True)
    hl1 = hits.query('volume_id ==8 & layer_id==2')
    hl2 = hits.query('volume_id ==8 & layer_id==4')
    hl3 = hits.query('volume_id ==8 & layer_id==6')
    hl4 = hits.query('volume_id ==8 & layer_id==8')
    print len(hl1), len(hl2), len(hl3), len(hl4)
    #print hl1
    # print hl2
    #print hits.loc[lambda h: h.volume_id ==8 & h.layer_id==2]
    #print hits[hits['volume_id']==8 & hits['layer_id']==2]
    nd = doublets(hits,hl1,hl2) #,dp12)
    hl1 = hits.query('volume_id ==8 & layer_id==2 & z<100 & z>-100')  # just those who can reach BPIX4
    print 'd12', nd # len(d12)
    nd = doublets(hits,hl2,hl3) #,dp23)
    hl2 = hits.query('volume_id ==8 & layer_id==4')
    print 'd23',nd # len(d12)
    nd = doublets(hits,hl3,hl4,0.07) #,dp34)
    hl3 = hits.query('volume_id ==8 & layer_id==6')
    print 'd34',nd # len(d12)
    
    CAThetaCut = 0.004 # 0.0010
    CAPhiCut = 0.175
    ptMin = 0.6 # algo tuned for 4T so need to scale up x2
    hardPtCut = 0.2
    region_origin_radius = 2.0
    ng=0
    ng2=0
    ng3=0
    nd=0
    nd2=0
    nd3=0
    na=0
    nok=0
    i1 =0
    nr=0
    hh1 = np.array([0,0],dtype=int)
    hh2 = np.array([0,0],dtype=int)
    hh3 = np.array([0,0],dtype=int)
    print 'hl1', len(hl1),len(hl1[hl1['tpt']>0.3])
    for h1 in hl1.iloc[1500:1600].itertuples() :
        ok=False
        i1+=1
        p1 = h1.particle_id
        if p1>10 & (h1.tpt>0.3) :
            nr+=1
#        print 'p1 ', p1
        hh1[0] = h1.up0
        hh1[1] = h1.up1
        xi = h1.x
        yi = h1.y
        ri = h1.r
        zi = h1.z
        szh2 = hl2.iloc[hh1[0]:hh1[1]].loc[lambda df : abs(zi-df['z'])<300]
#        if len(szh2)>0 : continue  # to time till here
        # print 'szh2', len(szh2)
        for h2 in szh2.itertuples() :
            hh2[0] = h2.up0
            hh2[1] = h2.up1
            szh3 = hl3.iloc[hh2[0]:hh2[1]].loc[lambda df : abs(h2.z-df['z'])<200]
            na += len(szh3)
            sh3 = szh3.loc[lambda df : areAlignedRZ(ri,zi,h2.r,h2.z,df['r'].values,df['z'].values,ptMin,CAThetaCut)]
            nd += len(sh3)
            sh33 = sh3.loc[lambda df : haveSimilarCurvature(xi,yi,h2.x,h2.y,df['x'].values,df['y'].values, ptMin, region_origin_radius, CAPhiCut, hardPtCut)]
            if len(sh33)>0 : continue   # to time till here
            nd2 += len(sh33)
            p2 = h2.particle_id
            if (p1==p2) : ng2+=1
            if (len(sh33)>0) & (p1==p2) :
                for p3 in sh33['particle_id'] :
                    if p3==p1 : ng3+=1
            for h3 in sh33.itertuples() :
                hh3[0] = h3.up0
                hh3[1] = h3.up1
                # hh3 = np.array([h3['up0'],h3['up1']],dtype=int)
                sh44 = hl4.iloc[hh3[0]:hh3[1]].loc[lambda df : abs(h3.z-df['z'])<200]\
                .loc[lambda df : areAlignedRZ(h2.r,h2.z,h3.r,h3.z,df['r'].values,df['z'].values,ptMin,CAThetaCut)]\
                .loc[lambda df : haveSimilarCurvature(h2.x,h2.y,h3.x,h3.y,df['x'].values,df['y'].values, ptMin, region_origin_radius, CAPhiCut, hardPtCut)]
                nd3 += len(sh44)
                p3 = h3.particle_id
                if (len(sh44)>0) & (p1==p2) & (p1==p3):
                    for p4 in sh44['particle_id'] :
                        if p4==p1 : 
                            ng+=1
                            ok=True
        if ok: nok+=1
        #print i1, 't123', na, nd, nd2, nd3, nr, ng2, ng3, ng,nok # len(d12)
    print 't123', na, nd, nd2, nd3, 'good', nr, ng2, ng3,ng,nok # len(d12)
  return evn

%prun -l 40 evn = runit()    
print 'done', evn

1000 120939
8892 7484 6672 6124
d12 2105965
d23 1588630
d34 911255
hl1 5408 4084
t123 2259964 5670 0 0 good 99 24 0 0 0
1001 93680
6737 5673 4987 4544
d12 1204007
d23 897960
d34 503383
hl1 3868 2912
t123 1240666 3034 0 0 good 97 43 0 0 0
 done 3
